<a href="https://colab.research.google.com/github/gabbum/vps-dicord-bot/blob/main/discord-bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Vps para discord bot ping ótimo**

In [ ]:
#@title Enviar dados para a vps

# Imports
import requests

#@markdown Google Drive:
Gdrive = True  # @param {type: 'boolean'}
global path

#@markdown Repositório da github:
repositoryUrl = "" #@param {type:"string"}

class MountClone:
    def __init__(
            self,
            Gdrive=False,
            repositoryUrl="",
            verbose=False):
        
        if Gdrive:
            self.mount_drive()
        if repositoryUrl:
            self.clone_repo(repositoryUrl, verbose)

    def __str__(self):
        try:
            return self.path
        except NameError:
            return ''
        
    def mount_drive(self):
        from google.colab import drive
        drive.mount('/content/drive')
        self.path = "/content/drive"

    def clone_repo(self, repositoryUrl, verbose=False):
        response = requests.get(repositoryUrl)
        if response.status_code == 200:
            print("✔️ O repositório é publico e foi sincronizado com sucesso!")
            ! cd /content
            ! git clone $repositoryUrl
            folder = repositoryUrl.split("/")[-1]
            self.path = f"/content/{folder}"
        else:
            print("❌ Repositório privado, não foi possivel sincronizar!")

if __name__ == "__main__":
    path = MountClone(Gdrive=Gdrive, repositoryUrl=repositoryUrl)

In [ ]:
# @title VSCode

# Imports
import os
import requests
import re
import shlex
import time
import signal

from subprocess import Popen, PIPE, call

# @markdown Senha: (Opcional)
Password = ""  # @param {type: 'string'}

#@markdown ATIVE ISSO, PARA A VPS NÃO DELIGAR RAPIDAMENTE
Sleep = True  # @param {type: 'boolean'}

class ColabCode:
    def __init__(
            self,
            port=10000,
            password='',
            verbose=False,
            sleep=False):

        self.install_server(verbose)
        self.start_code(port=port, password=password, verbose=verbose)

        if sleep:
            time.sleep(2000)
            print("Código rodando.")
            time.sleep(2000)
            print("Porque está vendo isso?")
            time.sleep(2000)
            print("Cara, é sério isso? Vc ainda ta lendo isso aq")
            time.sleep(2000)

    @staticmethod
    def __get_agro():
        hostname = None
        output = requests.get("http://localhost:45678/metrics").text
        result = re.search('cloudflared_tunnel_user_hostnames_counts{userHostname="https://(.+?)"}', output)
        if result is None:
            raise Exception("Cannot get any result from cloudflared metrics")
        hostname = result.group(1)
        if hostname is None:
            raise Exception("Cannot get the hostname from cloudflared, it looks like the connection has failed.")
        return hostname

    def install_server(self, verbose=False):
        os.system("kill -9 $(ps aux | grep 'cloudflared' | awk '{print $2}')")

        if not os.path.isfile("cloudflared"):
            ! wget -q -nc https://bin.equinox.io/c/VdrWdbjqyF/cloudflared-stable-linux-amd64.tgz &> /dev/null
            ! tar zxf cloudflared-stable-linux-amd64.tgz &> /dev/null
        else:
            if verbose:
                print("DEBUG: Skipping installation")
        
        if not os.path.isdir("/root/.local/share/code-server/"):
            settings_path = '/root/.local/share/code-server/User'
            ! curl -fsSL https://code-server.dev/install.sh | sh &> /dev/null
            if not os.path.exists(settings_path):
                os.makedirs(settings_path)
            with open(f'{settings_path}/settings.json', 'w') as settings:
                settings.write("""{
    "extensions.autoUpdate": false,
    "extensions.autoCheckUpdates": false
}""")

    def start_code(self, port=10000, password=None, verbose=False):
        if password:
            code_cmd = f'PASSWORD={password} code-server --port {port} --disable-telemetry'
        else:
            code_cmd = f'code-server --port {port} --auth none --disable-telemetry'

        agro_cmd = f'./cloudflared tunnel --url http://127.0.0.1:10000 --metrics localhost:45678'

        code_proc = Popen([code_cmd], shell=True, stdout=PIPE)
        agro_proc = Popen(shlex.split(agro_cmd), stdout=PIPE)

        if verbose:
            print(f"DEBUG: Code Server process: {code_proc.pid}")
            print(f"DEBUG: Cloudflared process: {agro_proc.pid}")

        time.sleep(3)
        try:
            info = self.__get_agro()
            print('Sua vps esta sincronizada com o repositorio/google drive, utilize ela aqui: https://{}/{}'.format(
                info, 
                f'?folder={path}' if path else ''))
        except NameError:
            info = self.__get_agro()
            print(f'Sua vps está pronta! Utilize-a aqui: https://{info}')
        except Exception:
            os.kill(code_proc.pid, signal.SIGKILL)
            os.kill(agro_proc.pid, signal.SIGKILL)
            print(f"Ooops, ocorreu erro inesperado.")
            sleep = False


if __name__ == "__main__":
    ColabCode(password=Password, sleep=Sleep)

# **Fim**

`Obrigado por utilizar nosso sistema, espero conseguir adicionar mais funções em breve!`
